### Mount Google Drive to runtime

In [1]:
from google.colab import drive
from os.path import join

# Mounting location on runtime for GDrive
ROOT = '/content/drive'

# Project workspace on GDrive
PROJECT_PATH = 'My Drive/Github'

# Mount GDrive on the runtime
drive.mount(ROOT)

# Create the full runtime project path and create a workspace at that location
WORKING_PATH = join(ROOT, PROJECT_PATH)
!mkdir "{WORKING_PATH}" 
%cd "{WORKING_PATH}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Github’: File exists
/content/drive/My Drive/Github


### Import libraries

In [2]:
import json
import string

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

f = open('/content/drive/My Drive/Github/mlteam4/sandbox/Amazon_Topic_Attributes_20200617061621.json')
topicDict = json.load(f)
f.close()

topics = list(topicDict.keys())

topicFeatures = []
labels = []

count = 0
for topic in topics:
    # Combine topic title and comments into one string
    title = topicDict[topic]['Topic Title']
    leadingComment = topicDict[topic]['Leading Comment']
    #otherComments = topicDict[topic]['Other Comments']
    
    commentList = [title] + [leadingComment] 
    featureString = ' '.join(commentList)

    # Replace newline and tab characters with spaces
    featureString = featureString.replace('\n', ' ')
    featureString = featureString.replace('\t', ' ')

    # Convert all letters to lowercase
    featureString = featureString.lower()

    #table = str.maketrans('', '', string.punctuation)
    #featureString = featureString.translate(table)

    featureString = featureString.encode(encoding='ascii', errors='ignore').decode('ascii')

    wordList = featureString.split()

    #wordList = [word for word in wordList if is_ascii(word)]

    wordList = [word.strip(string.punctuation) for word in wordList]

    wordList = [word for word in wordList if word.isalpha()]

    stop_words = set(stopwords.words('english'))
    wordList = [word for word in wordList if not word in stop_words]

    #table = str.maketrans('', '', string.punctuation)
    #wordList = [w.translate(table) for w in wordList]

    #print(wordList)

    '''
    tokens = word_tokenize(output)

    tokens = [w.lower() for w in tokens]

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    words = [word for word in stripped if word.isalpha()]
    '''

    featureString = ' '.join(wordList)
    if (featureString.strip() == ''):
        continue
    print(count)
    count += 1
    print(featureString)

    topicFeatures.append(featureString)

    # Append topic category to ground truth labels
    labels.append(topicDict[topic]['Category'])



vectorizer = CountVectorizer()
#vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(topicFeatures)

print(X.toarray())
print(X.shape)
#print(len(vectorizer.get_feature_names()))

#print(labels)

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1)
#X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=8)
#print(X_test.shape)

multinominalClassifier = MultinomialNB()
multinominalClassifier.fit(X_train, y_train)

numCorrect = (multinominalClassifier.predict(X_test) == y_test).sum()
accuracy = numCorrect / len(y_test)
print('Multinominal Classifier Accuracy: ' + str(accuracy))

gaussianClassifier = GaussianNB()
gaussianClassifier.fit(X_train.toarray(), y_train)

numCorrect = (gaussianClassifier.predict(X_test.toarray()) == y_test).sum()
accuracy = numCorrect / len(y_test)
print('Gausian Classifier Accuracy: ' + str(accuracy))



Streaming output truncated to the last 5000 lines.
cannot get response support error code hello friends valid upc code listing product board game created met codei needed go brand registry first brand registry requires uspto trademark brand applied trademark granted months due speed uspto need get products fba sooner im currently paying foreign storage costs wrote brand registry team explained situation told take photos product brand used box product sent several weeks ago since received nothing ive checked every days always get response checking specialists nothing moreand longer believe anyone successfully brand approved method long take think im going right way perhaps ive led astray amazons support team advice would appreciated thank
789
missing feedback anyone else trouble missing feedback reviews february year see several came april tried opening numerous cases emailed community help email address ideas thought maybe shut feedback due many shipping issues see new reviews arent fe

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
fba
8229
records indicate authorized access feature
8230
fba
8231
sku sku
8232
fba
8233
fba
8234
infant carrier astm cfr cfr sellers materials found hope get help thank infant carrier astm cfr cfr
8235
compare offers prices amazon compare offers prices amazon
8236
price sale price manufacturers suggested retail price
8237
shipping label shipping label
8238
listing
8239
free shipping eub kb
8240
linguistic services individual seller mx professional seller downgrade
8241
business addresspermanent addressmailing address tm
8242
vat service amazon vat service amazon
8243
still open amazon personal accounts still open amazon personal accounts
8244
coupons couponsasincoupons
8245
coupons coupons
8246
coupons coupons
8247
many illegal seller take advantage amazons bug made product show first page key words like seller besiva many asins seller like asins many review ads ebc appear first page search iphone x case strange anybody ways report kb 